# CDM IIIF Image Download

This script downloads images from Temple's digital collections.

OAI-PMH
https://digital.library.temple.edu/oai/oai.php?verb=ListSets\
https://digital.library.temple.edu/oai/oai.php?verb=ListRecords&metadataPrefix=oai_qdc
https://github.com/vphill/pyoaiharvester

Server?
https://server16002.contentdm.oclc.org/dmwebservices/index.php?q=dmGetCollectionList/json




# Install Packages


In [5]:
import urllib.request
import requests
import csv
import json
import pandas as pd
import os
from google.colab import files
%load_ext google.colab.data_table

# 1) Download Temple's Collection Manifest 

This manifest includes all info on Temple's CDM IIIF images

Select a Collection @id below to download it's relevant Images and Metadata

In [ ]:
# Provide CDM IIIF Manifest URL for Temple's digital collections
manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/manifest.json'

resp = urllib.request.urlopen(manifestUrl)
data = resp.read().decode("utf-8")
data = json.loads(data)

df = pd.json_normalize(data['collections'])
del df['@type']
df

,@id,label
0,https://cdm16002.contentdm.oclc.org/iiif/p1600...,Allied Posters of World War I
1,https://cdm16002.contentdm.oclc.org/iiif/p1600...,Ambler Campus History in Photographs
2,https://cdm16002.contentdm.oclc.org/iiif/p1503...,City Parks Association Photographs
3,https://cdm16002.contentdm.oclc.org/iiif/p1600...,Frank G. Zahn Railroad Photograph Collection
4,https://cdm16002.contentdm.oclc.org/iiif/p1600...,Franklin H. Littell Papers
5,https://cdm16002.contentdm.oclc.org/iiif/p1503...,George D. McDowell Philadelphia Evening Bullet...
6,https://cdm16002.contentdm.oclc.org/iiif/p1503...,George D. McDowell Philadelphia Evening Bullet...
7,https://cdm16002.contentdm.oclc.org/iiif/p2458...,Housing Association of the Delaware Valley Pho...
8,https://cdm16002.contentdm.oclc.org/iiif/p1503...,Impulse Dance Annuals
9,https://cdm16002.contentdm.oclc.org/iiif/p1600...,Jacob H. Gomborow Papers


## Create function to pull images from URLs

In [ ]:
pid = 0
img_urls = {}
img_smalls = {}


In [ ]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# 2) Download Collection-Level Metadata

Make this do iterative downloads for all collections

In [1]:
# provide the url to the collection
collectionURL = "https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll23/manifest.json"

In [6]:
resp1 = urllib.request.urlopen(collectionURL)
data1 = resp1.read().decode("utf-8")
data2 = json.loads(data1)
data2

HTTPError: ignored

In [ ]:
collection = data2['first']
responseManifest = requests.get(collection)
collManifest = json.loads(responseManifest.text)

In [ ]:
csv_out = csv.writer(open('collection.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject'])

44

In [ ]:
for i in collManifest['manifests']:
    pid = pid + 1   
    title = date = contributor = description = subject = ''
    manifestItemUrl = i['@id']
    responseManifestItem = requests.get(manifestItemUrl)
    manifestItem = json.loads(responseManifestItem.text)
    title = manifestItem['metadata'][0]['value']
    date = manifestItem['metadata'][1]['value']
    contributor = manifestItem['metadata'][2]['value']
    description = manifestItem['metadata'][3]['value']
    subject = manifestItem['metadata'][4]['value']
    filename = str(pid) + '.jpg'
    csv_out.writerow([title, date, contributor, description, subject, filename])    

## Other fields to query

https://researchworks.oclc.org/cdm-analyzer/cdm16002

creator


In [ ]:
df1 = pd.read_csv('collection.csv')
df1

,Title,Date,Contributor,Description,Subject
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"Dadd, Frank, 1851-1929, artist.; Straker Broth...",1914,"Signed ""Frank Dadd."" ""Poster No. 63(?)."" ""W TW...",1.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"Chorley & Pickersgill Ltd., printer.; Great Br...",[ca. 1914-1918,"Date based on dates of 1st World War. ""Poster ...",2.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"Straker Brothers Limited, printer.; Great Brit...",[ca. 1914-1918],"Date based on dates of 1st World War. ""Poster ...",3.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",Three questions to employers,scrc@temple.edu,"Bemrose & Sons, printer.; Great Britain. Parli...",[ca. 1914-1918],4.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"Johnson, Riddle & Co., printer.; Great Britain...",[ca. 1914-1918],"Date based on dates of 1st World War. ""Poster ...",5.jpg
...,...,...,...,...,...
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"United Cigar Stores Company, publisher.;",[1917 or 1918],"Initials ""NBT"" appear in a circle near bottom ...",435.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"Stanford's Geographical Estabt., publisher.; R...",[ca. 1914 -1918],Date based on dates of 1st World War. Orientat...,436.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"Flagg, James Montgomery, 1877-1960, artist.;",[1917 or 1918],"Signed ""James Montgomery Flagg."" Date based on...",437.jpg
"For Americans with Disabilities Act (ADA) accommodation, including help with reading this content, please contact diglib@temple.edu .",scrc@temple.edu,"J. Werner Ltd., publisher.;",[ca. 1914-1918],Date based on dates of 1st World War. &quot;No...,438.jpg


In [ ]:
files.download('collection.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#create function to search dataframe with string input through google form
# https://colab.research.google.com/notebooks/forms.ipynb
# https://stackoverflow.com/questions/11350770/filter-pandas-dataframe-by-substring-criteria
# https://www.geeksforgeeks.org/python-pandas-series-str-find/
# https://towardsdatascience.com/check-for-a-substring-in-a-pandas-dataframe-column-4b949f64852

# Iterate and get all metadata csv files

Close to working.

In [ ]:
def downallmetadata(df, pid = 0):
  pid = pid + 1   
  collection_title = str(pid) + '.csv'
  csv_out = csv.writer(open(collection_title, 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
  csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject'])
  for collectionUrl in df['@id']:
    resp1 = urllib.request.urlopen(collectionUrl)
    data1 = resp1.read().decode("utf-8")
    data2 = json.loads(data1)
    collection = pd.json_normalize(data2['first'])
    for url in collection['@id']:
      resp2 = urllib.request.urlopen(url)
      data3 = resp2.read().decode("utf-8")
      data4 = json.loads(data3)
      for i in data4['manifests']:
        title = date = contributor = description = subject = ''
        manifestItemUrl = i['@id']
        responseManifestItem = requests.get(manifestItemUrl)
        manifestItem = json.loads(responseManifestItem.text)
        title = manifestItem['metadata'][0]['value']
        date = manifestItem['metadata'][1]['value']
        contributor = manifestItem['metadata'][2]['value']
        description = manifestItem['metadata'][3]['value']
        subject = manifestItem['metadata'][4]['value']
        csv_out.writerow([title, date, contributor, description, subject]) 
             

In [ ]:
downallmetadata(df)

NotImplementedError: ignored

# 3) Download Images from specific Collection

This will download images from whichever collection you selected above

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path = "gdrive/MyDrive/images"
os.mkdir(path)


In [ ]:
image_dir = 'gdrive/MyDrive/images'

In [ ]:
def iiif_download(collectionURL, pid = 0):
  resp1 = urllib.request.urlopen(collectionURL)
  data1 = resp1.read().decode("utf-8")
  data2 = json.loads(data1)
  collection = pd.json_normalize(data2['first'])
  for url in collection['@id']:
    resp2 = urllib.request.urlopen(url)
    data3 = resp2.read().decode("utf-8")
    data4 = json.loads(data3)
    item = pd.json_normalize(data4['manifests'])
    for imageurl in item['@id']:
      resp3 = urllib.request.urlopen(imageurl)
      data5 = resp3.read().decode("utf-8")
      image = json.loads(data5)
      for jpg in image['sequences'][0]['canvases']:
          img_urls[pid]=jpg['images'][0]['resource']['service']['@id']
          filename = str(pid) + ".jpg"
          #imagename[pid] = item['label']
          #imagefile = str(imagename[pid]).replace(" ","").lower()
          # print(imagefile)
          #filename = imagefile[0:10] + ".jpg"
          turl = img_urls[pid]+"/full/pct:100/0/default.jpg"
          fullfilename = os.path.join(image_dir, filename)
          urllib.request.urlretrieve(turl, fullfilename)
          #files.download(filename)
          pid = pid +1
          if pid == 30:
                break

In [ ]:
iiif_download(collectionURL)

In [ ]:
!zip -r /content/IIIF.zip /content/

In [ ]:
files.download('/content/IIIF.zip')

# Download all TU CDM IIIF images

This function currently will get all images from all collections in Temple's ContentDM IIIF database.

In [ ]:
def downthemall(df, pid = 0):
  for collectionUrl in df['@id']:
    resp1 = urllib.request.urlopen(collectionUrl)
    data1 = resp1.read().decode("utf-8")
    data2 = json.loads(data1)
    collection = pd.json_normalize(data2['first'])
    for url in collection['@id']:
      resp2 = urllib.request.urlopen(url)
      data3 = resp2.read().decode("utf-8")
      data4 = json.loads(data3)
      item = pd.json_normalize(data4['manifests'])
      for imageurl in item['@id']:
        resp3 = urllib.request.urlopen(imageurl)
        data5 = resp3.read().decode("utf-8")
        image = json.loads(data5)
        for jpg in image['sequences'][0]['canvases']:
            img_urls[pid]=jpg['images'][0]['resource']['service']['@id']
            filename = str(pid) + ".jpg"
            #imagename[pid] = item['label']
            #imagefile = str(imagename[pid]).replace(" ","").lower()
            # print(imagefile)
            #filename = imagefile[0:10] + ".jpg"
            turl = img_urls[pid]+"/full/pct:100/0/default.jpg"
            urllib.request.urlretrieve(turl, filename)
            files.download(filename)
            pid = pid +1
            if pid == 100:
                  break

In [ ]:
downthemall(df)